In [1]:
# Import functions from these libraries

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from scipy.stats import norm
import statistics
import math
from statsmodels.stats.weightstats import ztest as ztest
sns.set_theme()

In [2]:
def parse_fasta_file(input_file):
    """This parses FASTA files and compiles them into a dictionary 
    with the identity tag as the keys
    and the nucleotide or amino acid sequences as the values
    """
    
    global parsed_seqs
    parsed_seqs = {}

    f = open(input_file)

    curr_seq_id = None
    curr_seq = []

    for line in f:

            line = line.strip()

            if line.startswith(">"):
                if curr_seq_id is not None:
                    parsed_seqs[curr_seq_id] = ''.join(curr_seq)

                curr_seq_id = line[1:]
                curr_seq = []
                continue

            curr_seq.append(line)

    parsed_seqs[curr_seq_id] = ''.join(curr_seq)
    
    print('Parse Complete')
    print('Dictionary name is: parsed_seqs')
    
    global reference_protein
    global reference_sequence
    
    reference_protein = list(parsed_seqs)[0]
    reference_sequence = list(parsed_seqs.values())[0]
    
    print('Reference sequence variable name is: reference_sequence')
    # separately, retrieve the first protein sequence in the dictionary
    # and display it as a reference protein for subsequence location
    
    return f'Reference Protein: {reference_protein}, {reference_sequence}'

In [3]:
def calculate_sequence_identity(seq1,seq2):
    """Return the level of sequence identity between seq1 and seq2"""

    if len(seq1) != len(seq2):
        raise ValueError(f"Aligned Sequences should be of equal length. Sequence 1 is {len(seq1)} nucleotides long but Sequence 2 is {len(seq2)} is nucleotides long.")
    
    matches = 0
    
    for i,residue in enumerate(seq1):

        if (i + 1) > len(seq2):
            continue

        residue2 = seq2[i]

        if residue.upper() == residue2.upper():
            matches += 1

    sequence_length = len(seq1)
    sequence_identity = matches/sequence_length
    
    print(sequence_identity)
    
    return sequence_identity

In [4]:
def calculate_subsequence_identity(seq1,seq2,loc1,loc2):
    """Return the level of subsequence identity between seq1 and seq2 in a frame bounded by loc1 and loc2"""

    if len(seq1) != len(seq2):
        raise ValueError(f"Aligned Subsequences should be of equal length. Sequence 1 is {len(seq1)} nucleotides long but Sequence 2 is {len(seq2)} is nucleotides long.")
        
    if loc1 == loc2:
        raise ValueError("Subsequence locations must be at least 1 nucleotide apart.")
        
    matches = 0
    
    subseq1 = seq1[loc1 - 1:loc2 - 1]
    subseq2 = seq2[loc1 - 1:loc2 - 1]
    # location numbers are converted from positive integers to their index equivalents
    # (e.g. the first residue of a subsequence is position 1)
    
    for i,residue in enumerate(subseq1):

        if (i + 1) > len(subseq2):
            continue

        residue2 = subseq2[i]

        if residue.upper() == residue2.upper():
            matches += 1

    subsequence_length = len(subseq1)
    subsequence_identity = matches/subsequence_length

    print(subsequence_identity)
    
    return subsequence_identity

In [5]:
def analyze_multiple_sequence_identities(parsed_seqs,loc1,loc2):
    """Find the sequence identity for every pairwise comparison of nucleotide or amino acid sequences
    and the identity for a subsequence beginning at loc1 and ending at loc2
    of the sequences in a fasta file
    then compile both into dictionaries with the comparison as the keys and the identity value as the values."""
    
    global sequence_identities
    global subsequence_identities
    
    sequence_identities = {}
    subsequence_identities = {}
    
    # open new dictionaries for the sequence and subsequence identities
    
    print("Calculating")
    
    location1 = loc1
    location2 = loc2
    
    sequences = parsed_seqs

    for identity_tag, sequence1 in sequences.items():
        # iterate over every sequence in your fasta file

        for identity_tag2, sequence2 in sequences.items():
            # compare with every sequence in your fasta file

            print(f"> Comparing {identity_tag} with {identity_tag2}")
            
            sequence_identity = calculate_sequence_identity(sequence1,sequence2)
            subsequence_identity = calculate_subsequence_identity(sequence1,sequence2,location1,location2)
            # return sequence and subsequence identities

            comparison = f"{sequence1}_{sequence2}"

            sequence_identities[comparison] = sequence_identity
            subsequence_identities[comparison] = subsequence_identity
            # compile into sequence and subsequence identity dictionaries
    
    print("Done")

In [6]:
def find_all(sequence,subsequence):
    """Find a predetermined subsequence (substring) in a longer sequence (string) including overlaps."""
    
    start = 0

    while True:
        start = sequence.find(subsequence,start)
        if start == -1: return
        yield start + 1 
        start += 1

In [7]:
def find_subsequence(sequence,subsequence):
    """Display the subsequence locations in a list
    and provide the subsequence length to be added
    to find the optimal reading frame."""
    
    subsequence_locations = list(find_all(sequence,subsequence))
    return f"Subsequence start locations: {subsequence_locations}", f"+{len(subsequence)}"

In [8]:
def sequence_comparison(sequence_identities,subsequence_identities):

    seq_id_list = list(sequence_identities.values())
    subseq_id_list = list(subsequence_identities.values())
    # move the sequence and subsequence identities from the dictionaries into lists

    mean = statistics.mean(seq_id_list)
    sd = statistics.stdev(seq_id_list)
    mean2 = statistics.mean(subseq_id_list)
    sd2 = statistics.stdev(subseq_id_list)

    print("Sequence identity mean:", mean)
    print("Sequence identity standard deviation:", sd)
    print("Subsequence identity mean:", mean2)
    print("Subsequence identity standard deviation:", sd2)
    # calculate the mean and standard deviation of the identities, then display

    if mean < mean2:
        print('Subsequence is more conserved than sequence')
    if mean > mean2:
        print('Subsequence is less conserved than sequence')
    if mean == mean2:
        print('Subsequence is as conserved as sequence')
    # clarify relationship of subsequence to sequence

    sns.displot(seq_id_list, x=seq_id_list, kind="kde", bw_adjust=1)
    plt.xticks([0, 0.2, 0.4, 0.6, 0.8, 1.0])
    plt.xlabel('Sequence Identity')
    # plot distribution of sequence identities
    
    sns.displot(subseq_id_list, x=subseq_id_list, kind="kde", bw_adjust=2)
    plt.xticks([0, 0.2, 0.4, 0.6, 0.8, 1.0])
    plt.xlabel('Subsequence Identity')
    # plot distribution of subsequence identities
    
    return stats.ttest_rel(subseq_id_list, seq_id_list), stats.ttest_1samp(subseq_id_list, mean)

In [9]:
parse_fasta_file('')

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
find_subsequence(reference_sequence,'')

In [ ]:
analyze_multiple_sequence_identities(parsed_seqs,,)

In [ ]:
sequence_comparison(sequence_identities,subsequence_identities)